In [ ]:
!pip install paho-mqtt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import paho.mqtt.client as mqtt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# MQTT broker information
mqtt_broker = "test.mosquitto.org"
mqtt_port = 1883
mqtt_topic = "nodered-elka"  # Set the MQTT topic to match the one used in Node-RED

# Constants for sound classification
num_classes = 3
feature_size = 1  # Modify this based on the number of features collected from Arduino
num_samples = 3

# Variables to store parsed data
features = []
labels = []

def on_connect(client, userdata, flags, rc):
    print("Connected to MQTT broker")
    client.subscribe(mqtt_topic)

def on_message(client, userdata, msg):
    message = msg.payload.decode()  # Decode the received message

    # Parse the received message and extract features and label
    # Modify this part based on the structure and format of your data
    # Assuming the received message is a comma-separated list of features followed by the label
    data = message.split(',')

    # Extract features
    feature_values = data[:feature_size]
    features.append(feature_values)

    # Extract label
    label = int(data[feature_size])
    labels.append(label)

    # Perform classification and printing the result
    if len(features) >= num_samples:
        # Convert the parsed data into numpy arrays
        X = np.array(features, dtype=np.float32)
        y = np.array(labels, dtype=np.uint8)

        # Convert the parsed data into numpy arrays
        X = np.array(features, dtype=np.float32)
        y = np.array(labels, dtype=np.uint8)

        # Preprocess the data (e.g., normalize feature values)
        X_normalized = X / np.max(X)

        # Reshape the data to match CNN input shape (num_samples, height, width, channels)
        X_normalized = X_normalized.reshape(-1, feature_size, 1, 1)

        # Convert labels to one-hot encoded vectors
        y_encoded = keras.utils.to_categorical(y, num_classes=num_classes)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=42)

        # Build the CNN model
        model = keras.Sequential([
            layers.Conv2D(16, kernel_size=(3, 1), activation="relu", input_shape=(feature_size, 1, 1)),
            layers.MaxPooling2D(pool_size=(2, 1)),
            layers.Flatten(),
            layers.Dense(64, activation="relu"),
            layers.Dense(num_classes, activation="softmax")
        ])

        # Compile the model
        model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

        # Train the model
        batch_size = 16
        epochs = 10

        # Training loop
        for epoch in range(epochs):
            history = model.fit(X_train, y_train, batch_size=batch_size, epochs=1, validation_data=(X_test, y_test))

        # Perform classification
        y_pred = model.predict(X_normalized)
        y_pred_labels = np.argmax(y_pred, axis=1)

        # Print the classification result
        class_names = ["Quiet", "Moderate", "Loud"]
        for pred_label in y_pred_labels:
            print("Classification Result:", class_names[pred_label])

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_broker, mqtt_port, 60)

# Start the MQTT loop
client.loop_start()

# Allow some time for data collection
# Adjust the duration as per your requirements
# Alternatively, you can define a specific number of data samples to collect
# and break the loop accordingly
import time
time.sleep(10)

# Stop the MQTT loop
client.loop_stop()


Exception in thread Thread-25 (_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/paho/mqtt/client.py", line 3591, in _thread_main
    self.loop_forever(retry_first_connection=True)
  File "/usr/local/lib/python3.10/dist-packages/paho/mqtt/client.py", line 1756, in loop_forever
    rc = self._loop(timeout)
  File "/usr/local/lib/python3.10/dist-packages/paho/mqtt/client.py", line 1164, in _loop
    rc = self.loop_read()
  File "/usr/local/lib/python3.10/dist-packages/paho/mqtt/client.py", line 1556, in loop_read
    rc = self._packet_read()
  File "/usr/local/lib/python3.10/dist-packages/paho/mqtt/client.py", line 2439, in _packet_read
    rc = self._packet_handle()
  File "/usr/local/lib/python3.10/dist-packages/paho/mqtt/client.py", line 3033, in

Connected to MQTT broker
